In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
#print(os.listdir("../input"))
# Any results you write to the current directory are saved as output.
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model
import glob
from scipy import misc
import tensorflow as tf
import numpy as np
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.layers import Dense, Activation, Dropout, Flatten,\
 Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.optimizers import Adam 

#Get back the convolutional part of a VGG network trained on ImageNet

Using TensorFlow backend.


In [2]:

model = Sequential()
# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11),
 strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())

In [3]:
# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

In [4]:
# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())


In [5]:
# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())


In [6]:
# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

In [7]:
# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())


In [9]:
# 3rd Dense Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())


In [10]:
# Output Layer
model.add(Dense(units=4))
model.add(Activation('softmax'))

batch_size = 4

# this is the augmentation configuration we will use for training
train_datagen = image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = image.ImageDataGenerator()


In [11]:

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
class_names = ['0','1','2','3']
train_generator = train_datagen.flow_from_directory(
        "./Images/Percentages_Set/Training",  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 150x150
        batch_size=batch_size,
          class_mode='categorical',
       )  # since we use binary_crossentropy loss, we need binary labels

# # this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
          "./Images/Percentages_Set/Validation",
          target_size=(224, 224),
          class_mode='categorical',
          batch_size=batch_size,
         )

test_generator = test_datagen.flow_from_directory(
          "./Images/Percentages_Set/Test",
          target_size=(224, 224),
          batch_size=1,
          class_mode=None,
          shuffle='False'
            )



Found 139 images belonging to 4 classes.
Found 52 images belonging to 4 classes.
Found 12 images belonging to 1 classes.


In [12]:
print(validation_generator.class_indices)

{'Crack0': 0, 'Crack1': 1, 'Crack2': 2, 'Non Crack': 3}


In [13]:
from keras import callbacks

log_dir = './tf-log/'
tb_cb = callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0)
cbks = [tb_cb]

In [ ]:


model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
filepath="weights.best.hdf5_2"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=2, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
#history = model.fit_generator(train_generator, epochs=10, steps_per_epoch=5, verbose=1,callbacks=callbacks_list, validation_data=validation_generator, validation_steps=5) #epochs=50, steps=2000
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
history =model.fit_generator(generator=train_generator,
                    steps_per_epoch=20,
                    validation_data=validation_generator,
                    validation_steps=20,
                    epochs=70,verbose=1)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Epoch 1/70
20/20 [==============================] - 32s 2s/step - loss: 2.2294 - acc: 0.3002 - val_loss: 7.9700 - val_acc: 0.5000
Epoch 2/70
20/20 [==============================] - 23s 1s/step - loss: 1.8779 - acc: 0.3167 - val_loss: 10.8461 - val_acc: 0.2750
Epoch 3/70
20/20 [==============================] - 23s 1s/step - loss: 1.8754 - acc: 0.3000 - val_loss: 8.4613 - val_acc: 0.3750
Epoch 4/70
20/20 [==============================] - 22s 1s/step - loss: 1.5489 - acc: 0.4002 - val_loss: 4.8462 - val_acc: 0.4375
Epoch 5/70
20/20 [==============================] - 27s 1s/step - loss: 1.7878 - acc: 0.3125 - val_loss: 1.7808 - val_acc: 0.5625
Epoch 6/70
20/20 [==============================] - 26s 1s/step - loss: 1.7648 - acc: 0.2666 - val_loss: 4.5819 - val_acc: 0.3625
Epoch 7/70
20/20 [==============================] - 22s 1s/step - loss: 1.8256 - acc: 0.2750 - val_loss: 4.2790 - val_acc: 0.4625
Epoch 8/70
20/20 [==============================] - 22s 1s/step - loss: 1.5976 - acc: 0.3

In [44]:
import sys
import os

target_dir = './models/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
model.save('./models/model.h5')
model.save_weights('./models/weights.h5')